In [1]:
import gurobipy as gp
import numpy as np

In [2]:
coefficient = dict()
# coefficient['y'] = -np.r_[[1.045], [0] * 10]
coefficient['y'] = np.array(-1.045)
coefficient['x'] = -np.r_[1.01:1.10:0.01]
# coefficient['b'] = np.r_[[1000], np.r_[[100] * 10]]
coefficient['b'] = np.array(1000)

In [3]:
from typing import Any

# 继承type，使Singleton成为元类，以Singleton为metaclass的类在实例化的时候调用Singleton的__call__()函数
class Singleton(type):
    _instance = {}

    def __call__(cls, *args: Any, **kwds: Any) -> Any:
        if cls not in cls._instance:
            cls._instance[cls] = super(Singleton, cls).__call__(*args,  **kwds)
        return cls._instance[cls]

In [4]:
from typing import Dict

class MP(metaclass=Singleton):
    def __init__(self, coefficient: Dict[str, np.ndarray]) -> None:
        self.master_problem = gp.Model("Master Problem")
        self.master_problem.setParam('OutputFlag', 0)
        self.y = self.master_problem.addMVar(1, vtype=gp.GRB.INTEGER, name="y")
        
        self.q = self.master_problem.addMVar(1, lb=-float("inf"), vtype=gp.GRB.CONTINUOUS, name="q")
        self.master_problem.setObjective(coefficient['y'] * self.y - 1 * self.q, sense=gp.GRB.MINIMIZE)
        self.b = coefficient['b']
        self.rhs = coefficient["b"] - self.y
    def update_constr(self, extrem_pnt: np.ndarray = None, extrem_ray: np.ndarray = None):

        if extrem_pnt:
            # 最优性约束
            self.master_problem.addConstr(extrem_pnt * self.rhs - self.q <= 0, name='optimal_constr')
        if extrem_ray:
            # 可行约束
            self.master_problem.addConstr(extrem_ray * self.rhs <= 0, name='feasible_constr')
    def get_lower_bound(self):
        self.master_problem.write('master_problem.lp')
        self.master_problem.optimize()
        return self.master_problem.ObjVal
    def get_y_value(self):
        return self.y.X

In [5]:
from typing import Dict

from networkx import constraint

class SUBP(metaclass=Singleton):
    def __init__(self, coefficient: Dict[str, np.ndarray]) -> None:
        self.sub_problem = gp.Model("Sub Problem")
        self.sub_problem.setParam('OutputFlag', 0)
        self.x = self.sub_problem.addMVar(coefficient['x'].shape, vtype=gp.GRB.CONTINUOUS, lb=0, ub=100, name="x")
        self.sub_problem.setObjective((coefficient['x'] * self.x).sum(),gp.GRB.MINIMIZE)
    def update_y_value(self, y_value: int):
        self.y_value = y_value
        cur_constraints = self.sub_problem.getConstrs()
        self.sub_problem.remove(cur_constraints)
        self.sub_problem.addConstr(self.x.sum() - 1000 + self.y_value <= 0, name='sub_problem_constr')
        # self.sub_problem.write('sub_problem.lp')
        
    def get_sub_pbm_res(self):
        self.sub_problem.setParam('InfUnbdInfo', 1)
        self.sub_problem.optimize()
        return self.sub_problem.Status

In [6]:
mp = MP(coefficient)
lb = mp.get_lower_bound()
print(f"Current lower bound is {lb}")
y_value = mp.get_y_value()
print(f"Current y is {y_value}")

Current lower bound is -2.045e+30
Current y is [1.e+30]


In [16]:
sp = SUBP(coefficient)
sp.update_y_value(y_value)
if sp.get_sub_pbm_res() == 2:
    # 子问题可行，更新原问题上界
    Pi_list = []
    for c in sp.sub_problem.getConstrs():
        Pi_list.append(-c.Pi)
    print(f"Current Pi is {Pi_list}")
else:
    # 子问题不可行，更新原问题上界
    farkasdual_list = []
    for c in sp.sub_problem.getConstrs():
        farkasdual_list.append(-c.farkasdual)
    print(f"Current farkasdual is {farkasdual_list}")

Current Pi is [1.1]


In [14]:
mp = MP(coefficient)
mp.update_constr(extrem_ray=farkasdual_list[0])
lb = mp.get_lower_bound()
print(f"Current lower bound is {lb}")
y_value = mp.get_y_value()
print(f"Current y is {y_value}")

Current lower bound is -1e+30
Current y is [1000.]


In [15]:
y_value

array([1000.])

In [24]:
mp.update_constr(extrem_pnt=-Pi_list[0])
lb = mp.get_lower_bound()
print(f"Current lower bound is {lb}")
y_value = mp.get_y_value()
print(f"Current y is {y_value}")
mp.master_problem.write('master_problem.lp')

Current lower bound is -1e+30
Current y is [1000.]


In [23]:
m_ = gp.read('master_problem.lp')
m_.optimize()

Read LP format model from file master_problem.lp
Reading time = 0.00 seconds
: 13 rows, 2 columns, 18 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 2 columns and 18 nonzeros
Model fingerprint: 0x79bb359c
Variable types: 1 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -1.00000e+30
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -1e+30 
No other solutions better than 0

Model is unbounded
Best objective -1.000000000000e+30, best bound -, gap -
